# How to record the protocol work with NCMC
### With a generalized Hamiltonian Monte Carlo (GHMC) propagator

Using a box of pure water as the test system.

In [1]:
from simtk import openmm, unit
from simtk.openmm import app
from openmmtools.testsystems import WaterBox
import numpy as np

# Setup box of water
size = 25.0*unit.angstrom     # The length of the edges of the water box.
temperature = 300*unit.kelvin
pressure = 1*unit.atmospheres
wbox = WaterBox(box_edge=size,nonbondedMethod=app.PME,cutoff=9*unit.angstrom,ewaldErrorTolerance=1E-5)
barostat = openmm.MonteCarloBarostat(pressure, temperature)
wbox.system.addForce(barostat)

3

# 1. Work to add/remove salt
It is straightforward to calculate the work to add and remove one salt pair in an otherwise pure box of water with the `saltswap` class. However, poor insertiona and deletion rates mean it is currently more practical the look at insetion and deletion seperately with the `MCMCSampler` class, discussed below.

This section looks at how to do this directly with `SaltSwap` and how to set the NCMC perturbation pathway

In [2]:
from saltswap import SaltSwap
from integrators import GHMCIntegrator

### Set the system up

In [3]:
# NCMC parameters
npert = 10     # The number of perturbation steps
nprop = 1        # The number of propagation steps per NCMC perturbation

delta_chem = 300*unit.kilojoule_per_mole # The chemical potential in multipes of kT
platform = 'OpenCL'   # The type of platform to run the dynamics with

# Create a compound inegrator
integrator = openmm.CompoundIntegrator()
# 1. The integrator for regular molecular dynamics
integrator.addIntegrator(openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 2.0*unit.femtoseconds))
# 2. The integrator for NCMC
integrator.addIntegrator(GHMCIntegrator(temperature, 1/unit.picosecond, 1.0*unit.femtoseconds, nsteps=nprop))

# Create the context
platform = openmm.Platform.getPlatformByName('OpenCL')
properties = {'OpenCLPrecision': 'mixed'}
context = openmm.Context(wbox.system, integrator, platform, properties)
context.setPositions(wbox.positions)
context.setVelocitiesToTemperature(temperature)

# Create the object to swap salt with water
swapper = SaltSwap(system=wbox.system, topology=wbox.topology,temperature=temperature, delta_chem=delta_chem,
                   integrator=integrator, pressure=pressure, npert=npert, nprop=nprop)

### The NCMC path

The NCMC perturbation (i.e. lambda) path is over the non bonded parameters of the two water molecules (specifically, two water oxyegens and four hydrogen atoms) that are converted into Na and Cl. The path is over the following dictionaries

In [4]:
a = swapper.wat2cat_parampath  # Water to cation path
b = swapper.wat2an_parampath   # Water to anion path
c = swapper.an2wat_parampath   # Anion to water path
d = swapper.cat2wat_parampath  # Cation to water path

Note that the code is flexible enough to allow for asymmetric paths, although the MCMC acceptence does not yet account for this.

Each path (e.g. `swapper.wat2cat_parampath`) is list of dictionaries of length equal to the number atoms in the water model. 

For Tip3p:

In [5]:
print len(swapper.wat2cat_parampath)

3


Each dictionary in the list specifies the NCMC path for one non-bonded parameter. There are 3 non-bonded parameters for each atom: the partial charge, Lennard-Jones $\sigma$, and Lennard-Jones $\epsilon$. 

The NCMC path for an **oxygen** atom to a cation is given by

In [6]:
print 'Partial charge', swapper.wat2cat_parampath[0]['charge'], '\n'
print 'LJ sigma', swapper.wat2cat_parampath[0]['sigma'], '\n'
print 'LJ epsilon', swapper.wat2cat_parampath[0]['epsilon']

Partial charge [-0.834, -0.6506, -0.4672, -0.2838, -0.10039999999999993, 0.08300000000000002, 0.26639999999999997, 0.4497999999999999, 0.6332000000000001, 0.8166, 1.0] 

LJ sigma [0.3150752406575124, 0.30796052659176115, 0.30084581252600995, 0.2937310984602587, 0.28661638439450743, 0.2795016703287562, 0.27238695626300496, 0.26527224219725376, 0.2581575281315025, 0.25104281406575124, 0.2439281] 

LJ epsilon [0.635968, 0.5811151299999999, 0.52626226, 0.47140938999999993, 0.41655652, 0.36170365, 0.30685078, 0.25199791000000005, 0.19714503999999997, 0.14229217, 0.0874393]


And similarly for the other atoms.

## Simulations

**IF** the NCMC protocol was efficient enough, the following would be sufficient to get the work to add and remove salt in an otherwise pure box of water.

In [7]:
for iteration in range(1):
    integrator.step(10)             # Run molecular dynamics for 10 steps. Setting very low for this example only
    swapper.update(context, nattempts=10, saltmax=1)      # Attempt 10 insertions or deletions

### Record the work

In [8]:
work_to_add_salt = swapper.work_add
work_to_remove_salt = swapper.work_rm

However, deletion rates are incredibly inefficient. To seperate insertions and deletions, it useful to seperate the insertion and deleletion simulations. 

The next section shows how to seperate insertions and deletions with `MCMCSampler` class.

# 2.1 Work to add salt with `MCMCSamper`
To calculate the work to **add** one anion and cation pair in a pure box of water, one must set the chemical potential to a value that ensures that an insertion move is _never_ accepted.

### Set-up the sampler class with a very high chemical potential

In [9]:
from mcmc_samplers import MCMCSampler

In [10]:
# Set up sampler class
npert = 10
nprop = 1
mdsteps = 50         # Setting very low for this example only
delta_chem = 10000   # To never insert
attempts = 10      # The number of insertion deletion attempts per iteration

timestep = 1.0*unit.femtoseconds
sampler = MCMCSampler(wbox.system, wbox.topology, wbox.positions, temperature=temperature, pressure=pressure,
                      npert=npert, nprop=nprop, propagator='GHMC', ncmc_timestep = timestep,
                      delta_chem=delta_chem, mdsteps=mdsteps, saltsteps=attempts, platform='OpenCL')

### NCMC path
The NCMC parameter paths are contained in the lists:

In [11]:
a = sampler.saltswap.wat2cat_parampath  # Water to cation path
b = sampler.saltswap.wat2an_parampath   # Water to anion path
c = sampler.saltswap.an2wat_parampath   # Anion to water path
d = sampler.saltswap.cat2wat_parampath  # Cation to water path

### Thermalize

In [12]:
# Thermalize
equilibration = 10    # Setting very low for this example only
sampler.gen_config(mdsteps=equilibration)

### Run insertion/deletion attempts

In [13]:
# With no molecular dynamics between each attempt
sampler.gen_label(saltsteps = attempts)

# With alternating steps of MD and MC.
sampler.multimove(nmoves = 10, mdsteps=mdsteps, saltsteps=attempts)
# This means: 10 iterations of MD and MC. Each iteration is comprised of 'mdsteps' of MD followed by 'attempts' of MC.

### Record the work

In [14]:
work_to_add_salt = sampler.saltswap.work_add

# 2.2 Work to remove salt with `MCMCSamper`
To calculate the work to **remove** one anion and cation pair in a box water that contains only one anion and cation, one must set the chemical potential to a value that ensures that an deletion move is _never_ accepted. This is after forcing the insertion of a salt pair.

### Set-up sampler class

In [15]:
# Set up sampler class
npert = 10
nprop = 1
steps = 50
delta_chem = -10000   # To insert straight away
attempts = 1          # Setting very low for this example only

timestep = 1.0*unit.femtoseconds
sampler = MCMCSampler(wbox.system, wbox.topology, wbox.positions, temperature=temperature, pressure=pressure,
                      npert=npert, nprop=nprop, propagator='GHMC', ncmc_timestep = timestep,
                      delta_chem=delta_chem, mdsteps=steps, saltsteps=attempts, platform='OpenCL')

### Add one molecule of salt

In [16]:
nosalt = True
while nosalt:
    sampler.saltswap.update(sampler.context, nattempts=1, cost=-10000)
    n_wats, n_ions, n_ions = sampler.saltswap.getIdentityCounts()
    nosalt = (n_ions == 0)

### Thermalize

In [17]:
equilibration = 10  # Setting very low for this example only
sampler.gen_config(mdsteps=equilibration)

### Run

In [18]:
sampler.multimove(nmoves = 10, mdsteps=mdsteps, saltsteps=attempts)

### Record the work

In [19]:
work_to_remove_salt = sampler.saltswap.work_rm